# H12 window size calibration

In [1]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
cohort_id = 'llineup'
#cohorts_analysis="20230223"
#contigs = ['2L']
sample_sets = ["1288-VO-UG-DONNELLY-VMF00219"]
sample_query = "sex_call == 'F'"
#min_cohort_size = 20
#max_cohort_size = 50
h12_calibration_contig = 'X'
use_gcs_cache = False
dask_scheduler = "threads"
window_sizes = (100, 200, 500, 1000, 2000, 5000, 10000, 20000)


In [2]:
pip install pyprojroot

Note: you may need to restart the kernel to use updated packages.


## Setup

In [3]:
import yaml
import pandas as pd
import malariagen_data
from pyprojroot import here
import numpy as np
import os
import dask
dask.config.set(scheduler=dask_scheduler);

In [4]:
af1 = malariagen_data.Af1(pre=True,
                          gcs_cache='/home/namulil/lstm_projects/funestus_llineup/gcs_cache',
                          results_cache='home/namulil/lstm_projects/funestus_llineup/results_cache'
                          )

In [5]:
contig = h12_calibration_contig


## Run calibration

In [6]:
calibration_runs = af1.h12_calibration(
    contig=h12_calibration_contig,
    analysis='funestus',
    sample_sets=sample_sets,
    sample_query=sample_query,
    min_cohort_size=None,
    max_cohort_size=None,
    window_sizes=window_sizes,
)
calibration_runs

<LazyLoader: 100, 1000, 10000, 200, 2000, 20000, 500, 5000>

In [7]:
## code create new variable window size that will return None if no selected window size is got
##iterates over all the window sizes craeates a value x form clalibration runs and sets on 95 percentiles of the x values
selected_window_size = None
for window_size in window_sizes:
    x = calibration_runs[str(window_size)]
    x95 = np.percentile(x, 95)
    if x95 < 0.1:
        selected_window_size = window_size
        break ## exit loop if appropriate window size is obtained
selected_window_size

500

## Write outputs

In [8]:
outdir = "/home/namulil/lstm_projects/funestus_llineup/notebooks/Signal_localisation"
os.makedirs(outdir, exist_ok=True)

In [9]:
## saving a dictionary of key (h12 window size) : value (selected window size)
output = {
    "h12_window_size": selected_window_size
}
with open(os.path.join(outdir, f"{cohort_id}.yaml"), mode="w") as output_file:
    yaml.safe_dump(output, output_file)
    

## doing as above for the X chromosome###


In [10]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
#cohort_id = 'llineupx'
#cohorts_analysis="20230223"
#contigs = ['2L']
#sample_sets = ["1288-VO-UG-DONNELLY-VMF00219"]
#sample_query = "sex_call == 'F'"
#min_cohort_size = 20
#max_cohort_size = 50
#h12_calibration_contig = 'X'
#use_gcs_cache = False
#dask_scheduler = "threads"
#window_sizes = (100, 200, 500, 1000, 2000, 5000, 10000, 20000)

In [ ]:
#calibration_runs = af1.h12_calibration(
    #contig=h12_calibration_contig,
    #analysis='funestus',
    #sample_sets=sample_sets,
    #sample_query=sample_query,
    #min_cohort_size=None,
    #max_cohort_size=None,
    #window_sizes=window_sizes,
#)
#calibration_runs

In [12]:
## code create new variable window size that will return None if no selected window size is got
##iterates over all the window sizes craeates a value x form clalibration runs and sets on 95 percentiles of the x values
#selected_window_size = None
#for window_size in window_sizes:
    #x = calibration_runs[str(window_size)]
    #x95 = np.percentile(x, 95)
    #if x95 < 0.1:
        #selected_window_size = window_size
        #break ## exit loop if appropriate window size is obtained
#selected_window_size

500

In [13]:
## saving a dictionary of key (h12 window size) : value (selected window size)
#output = {
   # "h12_window_size": selected_window_size
#}
#with open(os.path.join(outdir, f"{cohort_id}.yaml"), mode="w") as output_file:## open file for writing dictionary in YAML format and writtable
 #   yaml.safe_dump(output, output_file)## dump the created dictionary into the output file